In [1]:
#webscrap, import libs
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
#function to get page html
def getsoup(my_url):
    #download webpage, raw html
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()

    #html parse
    page_soup = soup(page_html, "html.parser")
    containers = page_soup.findAll("div", {"class":"product-pod--padding"})
    return page_soup

In [3]:
#function to get each tag value (brand, name, rating...)
def gettagvalue (tag, tag_class, tag_class_name): 
    while True:
        try:
            field = container.findAll(tag, {tag_class:tag_class_name})
            field_value = field[0].text
            break
        except IndexError:
            field_value = "missing"
            break
    return field_value

In [4]:
#function to get tag value for StarRating
def getstarvalue (tag, tag_class, tag_class_name): 
    while True:
        try:
            field = container.findAll(tag, {tag_class:tag_class_name})
            field_value = field[0]["style"]
            break
        except IndexError:
            field_value = "missing"
            break
    return field_value

In [5]:
#function to get tag value for specs
def getspecsvalue (tag, tag_class, tag_class_name): 
    while True:
        try:
            field = container.findAll(tag, {tag_class:tag_class_name})
            n_cat=len(field)
            field_value=""
            for i in range(n_cat):
                each_value=field[i].text
                field_value=field_value+each_value+ "|"
            break
        except IndexError:
            field_value = "missing"
            break
    return field_value

In [6]:
#function for dataframe and export to CSV
def product_export_table_and_print(data):
    table = pd.DataFrame(data, columns=['Brand', 'Name', 'ModelID', 'Star','Rating','Price','ProductURL','Spec_Category','Spec_Value'])
    table.index = table.index + 1
    table.to_csv(f'C:/Users/Maria/Documents/{mass_merchant}_{category}.csv',
                 sep=',', encoding='utf-8', index=False)
    return table

In [7]:
def detailspec_export_table_and_print(data):
    table = pd.DataFrame(data, columns=['Model', 'DetailSpec'])
    table.index = table.index + 1
    table.to_csv(f'C:/Users/Maria/Documents/{mass_merchant}_{category}_fullspecs.csv',
                 sep=',', encoding='utf-8', index=False)
    return table

In [8]:
def detailspecs(product_page_soup):
    models = product_page_soup.findAll("h2", {"class":"product-info-bar__detail--24WIp"})
    model = models[1].text

    product_containers = product_page_soup.findAll("div", {"class":"specifications__row col__12-12 col__12-12--xs col__12-12--sm col__6-12--md"})
    for product_container in product_containers:
        detailspecs = product_container.findChildren("div")
        for spec in detailspecs:
            fullspecdata['Model'].append(model)
            fullspecdata['DetailSpec'].append(spec.text)

### Specify Mass Merchant, Product Category, Number of Pages

In [13]:
#######  INPUTS  ############
# mass_merchant='homedepot'
# category='LaundaryDryer'
# my_url = 'https://www.homedepot.com/b/Appliances-Washers-Dryers-Dryers/N-5yc1vZc3p3'
# num_of_pages=19

#-----------------------------
# mass_merchant='homedepot'
# category='LaundaryWasher'
# my_url = 'https://www.homedepot.com/b/Appliances-Washers-Dryers-Washing-Machines/N-5yc1vZc3ov#Shop%20All'
# num_of_pages=13

#-----------------------------

#mass_merchant='homedepot'
#category='FrontLoadWasher'
#my_url = 'https://www.homedepot.com/b/Appliances-Washers-Dryers-Washing-Machines-Front-Load-Washers/N-5yc1vZc3pj'
#num_of_pages=8

# mass_merchant='homedepot'
# category='DualFuelRange'
# my_url = 'https://www.homedepot.com/b/Appliances-Ranges-Dual-Fuel-Ranges/N-5yc1vZc3q2'
# num_of_pages=9


#-----------------------------

#Bosch
#my_url = 'https://www.homedepot.com/b/Appliances-Washers-Dryers/Bosch/N-5yc1vZc3olZ9u?storeSelection='

#-----------this one will run out of range, run 5 pages at once------------------
mass_merchant='homedepot'
category='Dishwasher'
my_url='https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts'
num_of_pages=17


### Then run the below

In [14]:
#Execute Main Body
productlist = {
    'Brand': [],
    'Name': [],
    'ModelID': [],
    'Star': [],
    'Rating': [],
    'Price': [],
    'ProductURL': [],
    'Spec_Category': [] ,
    'Spec_Value': []  
}

fullspecdata = {
    'Model': [],
    'DetailSpec': []
}

for i in range(num_of_pages):
    page_soup = getsoup(my_url)
    brand_tag = 'span'
    brand_tag_class = 'class'
    brand_tag_class_name = 'product-pod__title__brand--bold'
    name_tag = 'span'
    name_tag_class = 'class'
    name_tag_class_name = 'product-pod__title__product'
    modelID_tag= 'div'
    modelID_tag_class= 'class'
    modelID_tag_class_name= 'product-pod__model'
    star_tag= 'span'
    star_tag_class='class'
    star_tag_class_name='stars'
    rating_tag='span'
    rating_tag_class='class'
    rating_tag_class_name='product-pod__ratings-count'
    price_tag='div'
    price_tag_class='class'
    price_tag_class_name='price-format__main-price'
    spec_cat_tag="div"
    spec_cat_tag_class="class"
    spec_cat_tag_class_name="kpf__name kpf__name--simple kpf__name--one-column"
    spec_val_tag="div"
    spec_val_tag_class="class"
    spec_val_tag_class_name="kpf__value kpf__value--simple kpf__value--one-column"

    containers = page_soup.findAll("div", {"class":"product-pod--padding"})
    for container in containers:
        brand_value = gettagvalue(brand_tag,brand_tag_class,brand_tag_class_name)
        name_value = gettagvalue(name_tag, name_tag_class,name_tag_class_name)
        modelID_value = gettagvalue(modelID_tag,modelID_tag_class,modelID_tag_class_name)
        star_value = getstarvalue(star_tag,star_tag_class,star_tag_class_name)
        rating_value = gettagvalue(rating_tag,rating_tag_class,rating_tag_class_name)
        price_value = gettagvalue(price_tag,price_tag_class,price_tag_class_name)
        product_url = "https://www.homedepot.com" + str(container.a['href'])
        spec_cat = getspecsvalue(spec_cat_tag,spec_cat_tag_class,spec_cat_tag_class_name)
        spec_val = getspecsvalue(spec_val_tag,spec_val_tag_class,spec_val_tag_class_name)
        

        productlist['Brand'].append(brand_value)
        productlist['Name'].append(name_value)
        productlist['ModelID'].append(modelID_value)
        productlist['Star'].append(star_value)
        productlist['Rating'].append(rating_value)
        productlist['Price'].append(price_value)
        productlist['ProductURL'].append(product_url)
        productlist['Spec_Category'].append(spec_cat)
        productlist['Spec_Value'].append(spec_val)
        
        product_page_soup = getsoup(product_url)
        detailspecs(product_page_soup)
    
    #next page 
    pages = page_soup.findAll("li", {"class":"hd-pagination__item hd-pagination__button"}) 
    for page in pages:
        my_url = "https://www.homedepot.com" + str(page.a["href"])
    print("Next Page URL: " + my_url)
    i+=1
    page_soup = getsoup(my_url)

Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.homedepot.com/b/Appliances-Dishwashers/N-5yc1vZc3po?NCNI-5&catStyle=ShowProducts
Next Page URL: https://www.h

In [15]:
#Print out data and save to CSV
product_export_table_and_print(productlist)

,Brand,Name,ModelID,Star,Rating,Price,ProductURL,Spec_Category,Spec_Value
1,Samsung,24 in. Stainless Steel Top Control Built-In Ta...,Model# DW80R2031US,width:75.96199999999999%,(1253),missing,https://www.homedepot.com/p/Samsung-24-in-Stai...,Tub Material|Sound Rating Range (Decibels)|Hei...,Plastic Tub|Quiet|35.125|24 In.|Top Control|
2,Bosch,300 Series 24 in. Stainless Steel Front Contro...,Model# SHEM63W55N,width:91.368%,(3524),missing,https://www.homedepot.com/p/Bosch-300-Series-2...,Tub Material|Sound Rating Range (Decibels)|Hei...,Stainless Steel Tub|Quietest|35|24 In.|Front C...
3,Bosch,500 Series 24 in. Stainless Steel Top Control ...,Model# SHPM65Z55N,width:92.418%,(3165),missing,https://www.homedepot.com/p/Bosch-500-Series-2...,Tub Material|Sound Rating Range (Decibels)|Hei...,Stainless Steel Tub|Quietest|35.0625|24 In.|To...
4,Whirlpool,24 in. Monochromatic Stainless Steel Front Con...,Model# WDF520PADM,width:84.62400000000001%,(12014),missing,https://www.homedepot.com/p/Whirlpool-24-in-Mo...,Tub Material|Sound Rating Range (Decibels)|Hei...,Plastic Tub|Quiet|34.5|24 In.|Front Control|
5,LG Electronics,24 in. Stainless Steel Front Control Dishwashe...,Model# LDFN454HT,width:100%,(4),missing,https://www.homedepot.com/p/LG-Electronics-24-...,Tub Material|Sound Rating Range (Decibels)|Hei...,Stainless Steel Tub|Quieter|33.6|24 In.|Front ...
...,...,...,...,...,...,...,...,...,...
200,Frigidaire,24 in. Stainless Steel Front Control Built-In ...,Model# FFCD2413US,width:79.34400000000001%,(5027),missing,https://www.homedepot.com/p/Frigidaire-24-in-S...,Tub Material|Sound Rating Range (Decibels)|Hei...,Plastic Tub|Average|35|24 In.|Front Control|
201,KitchenAid,24 in. PrintShield Stainless Steel Top Control...,Model# KDTM404KPS,width:93.78800000000001%,(2231),missing,https://www.homedepot.com/p/KitchenAid-24-in-P...,Tub Material|Sound Rating Range (Decibels)|Hei...,Stainless Steel Tub|Quietest|34.63|24 In.|Top ...
202,LG Electronics,24 in. PrintProof Stainless Steel Top Control ...,Model# LDT5678SS,width:83.094%,(763),missing,https://www.homedepot.com/p/LG-Electronics-24-...,Tub Material|Sound Rating Range (Decibels)|Hei...,Stainless Steel Tub|Quieter|33.60|24 In.|Top C...
203,Whirlpool,24 in. Fingerprint Resistant Stainless Steel T...,Model# WDT970SAHZ,width:89.10199999999999%,(3606),missing,https://www.homedepot.com/p/Whirlpool-24-in-Fi...,Tub Material|Sound Rating Range (Decibels)|Hei...,Stainless Steel Tub|Quieter|34.5|24 In.|Top Co...


In [17]:
#detailspec_export_table_and_print(fullspecdata)